### Playnomore
- 가방, 옷, 악세사리 판매하는 쇼핑몰
- http://playnomore.co.kr/

In [1]:
import scrapy
import requests
from scrapy.http import TextResponse
from fake_useragent import UserAgent

In [2]:
# 1. 프로젝트 생성

In [3]:
!scrapy startproject playnomore

Error: scrapy.cfg already exists in /Users/jhkim/Documents/dev/TIL_/playnomore


In [4]:
# 2. items.py 코드 작성

In [5]:
%%writefile playnomore/playnomore/items.py
import scrapy

class PlaynomoreItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    img = scrapy.Field()
    link = scrapy.Field()

Overwriting playnomore/playnomore/items.py


In [6]:
# 3. xpath 찾기 : 링크, 상품 데이터

In [7]:
# 링크

In [8]:
url = "http://playnomore.co.kr/category/bag/24/"
# headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
headers = {"User-Agent": UserAgent().chrome}
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 http://playnomore.co.kr/category/bag/24/>

In [9]:
links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div[1]/a/@href').extract()
links = list(map(lambda data:response.urljoin(data), links))
links[:3]

['http://playnomore.co.kr/product/detail.html?product_no=573&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=572&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=550&cate_no=24&display_group=1']

In [10]:
# 상세 데이터

In [11]:
url = "http://playnomore.co.kr/product/pre-order20off-micro-baguette-yellow-180/573/?cate_no=24&display_group=1"
headers = {"User-Agent": UserAgent().chrome}
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 http://playnomore.co.kr/product/pre-order20off-micro-baguette-yellow-180/573/?cate_no=24&display_group=1>

In [12]:
title1 = response.xpath(
    '//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()')[0].extract()
title2 = response.xpath(
    '//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
title = title1 + " ".join(title2)
price = response.xpath(
    '//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()')[0].extract()
img = "http:" + response.xpath(
    '//*[@id="contents"]/div[1]/div[1]/div[1]/div[2]/ul/li[1]/img/@src')[0].extract()

title, price, img

('[Pre-Order/20%off]  MICRO BAGUETTE  yellow ',
 '$ 144',
 'http://playnomore.co.kr/web/product/small/20200407/43ec3b61d30531a05a3d727780c9b1f3.jpg')

In [13]:
# 4. spider.py 코드 작성 : fake user agent 설정

In [14]:
!pip install scrapy-fake-useragent

In [21]:
%%writefile playnomore/playnomore/spiders/spiders.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = "Playnomore"
    custom_settings = {
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadmiddlewares.useragent.UserAgentMiddleware": None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
        }
    }
    
    def __init__(self,category1, category2, **kwargs): #스크래피 실행시 아규먼트 설정
        self.start_urls = ["http://palynomore.co.kr/category/{}/{}".format(category1, category2)]#스크래피 실행시 아규먼트 설정
        super.__init__(**kwargs)#스크래피 실행시 아규먼트 설정
    
    def start_requests(self):
        self.start_urls = ["http://playnomore.co.kr/category/bag/24/"]#스크래피 실행시 아규먼트 설정
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)
    
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div[1]/a/@href').extract()
        links = list(map(lambda data:response.urljoin(data), links))
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()')[0].extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item["title"] = title1 + " ".join(title2)
        item["price"] = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()')[0].extract()
        item["img"] = "http:" + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[2]/ul/li[1]/img/@src')[0].extract()
        item["link"] = response.url
        yield item

Overwriting playnomore/playnomore/spiders/spiders.py


In [16]:
# 5. 스크래피 실행시 아규먼트 설정

In [ ]:
# bag/24/
# shoes/25/
# beauty/28/

In [17]:
# 6. 스크레피 실행

In [18]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore.csv

Writing run.sh


In [19]:
!source run.sh

2020-07-10 14:59:32 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: playnomore)
2020-07-10 14:59:32 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug 13 2019, 15:17:50) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Darwin-19.5.0-x86_64-i386-64bit
2020-07-10 14:59:32 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'playnomore', 'FEED_FORMAT': 'csv', 'FEED_URI': 'playnomore.csv', 'NEWSPIDER_MODULE': 'playnomore.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['playnomore.spiders']}
2020-07-10 14:59:32 [scrapy.extensions.telnet] INFO: Telnet Password: 3f2d30d7da119219
2020-07-10 14:59:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter

In [20]:
pd.read_csv("playnomore/playnomore.csv")

,img,link,price,title
0,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO ice grey
1,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO black
2,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO mint
3,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO candy pink
4,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO blue
5,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO whtie
6,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO green
7,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/new-open-event...,$ 162,[New Open Event 10%] RAINBOW HOBO red
8,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/15off-micro-ca...,$ 153,[15%off] MICRO CANDY magenta
9,http://playnomore.co.kr/web/product/small/2020...,http://playnomore.co.kr/product/pre-order20off...,$ 144,[Pre-Order/20%off] MICRO BAGUETTE yellow
